In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import pymongo

from time import time

#matplotlib.rcParams.keys()
matplotlib.rcParams['font.size'] = 11

import pickle
from bson.binary import Binary

In [2]:
# helper functions

# Complex morlet wavelets (cmw) consist of a sine wave multiplied by a Gaussian. 
# (The imaginary term comes from Euler's formula.)
cmw = lambda t, f, s: (1/np.sqrt(s*np.sqrt(np.pi))) * np.exp(-t**2/(2*s**2)) * np.exp(1j*2*np.pi*f*t)
# Where t is time axis, f is peak frequency of wavelet, and s is standard deviation of Gaussian (std_for_gaussian)
std_for_gaussian = lambda n, f: n/(2*np.pi*f)
# Where f is peak frequency of wavelet and n is number of wavelet cycles

# Although not used in this notebook, this visualize_wavelet function can be helpful for understanding.
def visualize_wavelet(freq=6, n_wavelet_cycles=6, windowLengthSeconds=1, fs=300):
    x = np.linspace(-windowLengthSeconds/2, windowLengthSeconds/2, int(windowLengthSeconds*fs)+1)  # 300 Hz sampling rate
    complex_morlet_wavelet = cmw(x, freq, std_for_gaussian(n=n_wavelet_cycles, f=freq))
    print('x.shape:',x.shape)
    print('complex_morlet_wavelet.shape:', complex_morlet_wavelet.shape)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot(x, np.real(complex_morlet_wavelet), np.imag(complex_morlet_wavelet))
    ax.set_xlabel('time(s)')
    ax.set_ylabel('real')
    ax.set_zlabel('imag')
    plt.show()

    fig = plt.figure()  # real component is centered at time=0 (like a cosine function)
    ax = fig.gca(projection='3d')
    ax.plot(x, np.real(complex_morlet_wavelet), np.imag(complex_morlet_wavelet))
    ax.view_init(90, -90)
    ax.set_xlabel('time(s)')
    ax.set_ylabel('real')
    ax.set_zlabel('imag')
    plt.show()

    fig = plt.figure()   # imaginary component is phase-shifted by 90 degrees (like a sine function)
    ax = fig.gca(projection='3d')
    ax.plot(x, np.real(complex_morlet_wavelet), np.imag(complex_morlet_wavelet))
    ax.view_init(0, -90)
    ax.set_xlabel('time(s)')
    ax.set_ylabel('real')
    ax.set_zlabel('imag')
    plt.show()

# Start MongoDB

In [3]:
pymongo.__version__

'3.4.0'

In [4]:
client = pymongo.MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
db = client.test_database
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

In [6]:
# test whether I can access database from previous test session
db.test_capped3.find_one()

{'_id': ObjectId('60c0003ed64ad42fe4435f7b'),
 't': 1.75,
 'sin_A': 2.3516932427448712e-14,
 'sin_B': -9.809554005910593e-16,
 'sin_C': 2.3516932427448712e-14,
 'sin_D': -9.809554005910593e-16}

In [7]:
#db.create_collection('raw_cappedCollection_65', capped=True, size=100000, max=65)

CollectionInvalid: collection raw_cappedCollection_65 already exists

In [7]:
db.collection_names()

['test_capped2',
 'test_capped',
 'raw_cappedCollection_65',
 'posts',
 'test_capped3',
 'processed_0']

In [9]:
#db.create_collection('processed_0')
#db.collection_names()

CollectionInvalid: collection processed_0 already exists

In [8]:
def plot_finished_session(list_wavelet_solutions_, list_Hjorth_parameters_):
    complex_wavelet_tensor = np.stack(list_wavelet_solutions_, axis=0)
    
    print('plotting wavelet power on each channel (columns) and each frequency (rows)')
    power_tensor = np.abs(complex_wavelet_tensor**2)
    
    n_time_points = complex_wavelet_tensor.shape[0]
    rows = complex_wavelet_tensor.shape[1]
    columns = complex_wavelet_tensor.shape[2]
    t = np.linspace(0, 0.25*n_time_points, n_time_points, endpoint=False)
    fig, axes = plt.subplots(rows, columns, figsize=(columns*5, rows*3))
    for row in rows:
        for col in columns:
            axes[row, column].plot(t, power_tensor[:, row, column])
    plt.show()
    
    print('plotting Hjorth parameters')
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))
    for ax, param in zip(axes,['A','M','C']):
        ax.plot(t, [Hjorth_params[param] for Hjorth_params in list_Hjorth_parameters_])
        ax.set_title(param)
    plt.show()       
            

In [9]:
# Using Complex Morlet Wavelets
cmw = lambda t, f, s: (1/np.sqrt(s*np.sqrt(np.pi))) * np.exp(-t**2/(2*s**2)) * np.exp(1j*2*np.pi*f*t)
std_for_gaussian = lambda n, f: n/(2*np.pi*f)

windowLengthSeconds=0.25
fs=256
x = np.linspace(-windowLengthSeconds/2, windowLengthSeconds/2, int(windowLengthSeconds*fs)+1)  
wavelet_freq_peaks = [6, 12, 24] #range(5,80,2)
complex_morlet_wavelet = np.vstack([cmw(x, f=f_, s=std_for_gaussian(n=f_/5, f=f_)) for f_ in wavelet_freq_peaks])

In [10]:
"""
### Code adapted from:
**********************
Mind Monitor - Minimal EEG OSC Receiver
Coded: James Clutterbuck (2021)
Requires: pip install python-osc
**********************
"""
from datetime import datetime
from pythonosc import dispatcher
from pythonosc import osc_server

ip = "192.168.1.165"
port = 5000

#EEG_log_list = []
#global count_time_point
count_time_point = 0

def eeg_handler(address: str,*args):
    global count_time_point
    
    count_time_point = count_time_point + 1
    dateTimeObj = datetime.now()
    printStr = dateTimeObj.strftime("%Y-%m-%d %H:%M:%S.%f")
    data_dict = {"t":printStr, "TP9":float(args[0]), "AF7":float(args[1]), "AF8":float(args[2]), "TP10":float(args[3])}
    
    db.raw_cappedCollection_65.insert_one(data_dict)
    
    if count_time_point == 64:  # if a quarter-second of data has been collected
        print('finished loop of quarter second')
        count_time_point = 0
        docs = db.raw_cappedCollection_65.find({}, projection={"TP9":1, "AF7":1, "AF8":1, "TP10":1, "_id":0})
        X = np.array([[doc['TP9'], doc['AF7'], doc['AF8'], doc['TP10']] for doc in docs])
        wavelet_solution = np.dot(complex_morlet_wavelet, X)   # Convolution-like operation. Shape=(n_wavelets, n_channels)
        Hjorth_parameters = {'A': np.var(X, axis=0), 
                             'M': np.sqrt(np.var(np.diff(X, axis=0), axis=0)/np.var(X, axis=0)),
                             'C': np.sqrt(np.var(np.diff(X, n=2, axis=0), axis=0)/np.var(X, axis=0))}
        
        #list_wavelet_solutions.append(wavelet_solution)
        #list_Hjorth_parameters.append(Hjorth_parameters)
        
        processed_data_dict = {'complex_wavelet_matrix':wavelet_solution, 
                              'Hjorth_A':Hjorth_parameters['A'],
                              'Hjorth_M':Hjorth_parameters['M'],
                              'Hjorth_C':Hjorth_parameters['C']}  #Binary(pickle.dumps(wavelet_solution))
        
        db.processed_0.insert_one(processed_data_dict)

    
    #for arg in args:
    #    printStr += ","+str(arg)
    #print(printStr)
    #EEG_log_list.append(printStr)
    #print('EEG LOG LIST NOW HAS LENGTH', len(EEG_log_list))
    
if __name__ == "__main__":
    dispatcher = dispatcher.Dispatcher()
    dispatcher.map("/muse/eeg", eeg_handler)

    server = osc_server.ThreadingOSCUDPServer((ip, port), dispatcher)
    print("Listening on UDP port "+str(port))
    server.serve_forever()

Listening on UDP port 5000


KeyboardInterrupt: 

In [11]:
list_wavelet_solutions = [x for x in db.processed_0.find({}, projection={'complex_wavelet_matrix':1, '_id':0})]

list_Hjorth_parameters = [{'A':doc['Hjorth_A'], 'M':doc['Hjorth_M'], 'C':doc['Hjorth_C']} for doc in db.processed_0.find({}, 
                                       projection={'Hjorth_A':1, 'Hjorth_M':1, 'Hjorth_C':1, '_id':0})]

In [14]:
for doc in db.raw_cappedCollection_65.find():
    print(doc)

{'_id': ObjectId('60c019b9d64ad447d49253ef'), 't': '2021-06-08 21:30:33.879988', 'TP9': 797.8021850585938, 'AF7': 807.4725341796875, 'AF8': 804.6520385742188, 'TP10': 794.5787353515625}
{'_id': ObjectId('60c019b9d64ad447d49253f0'), 't': '2021-06-08 21:30:33.879988', 'TP9': 807.87548828125, 'AF7': 808.6813354492188, 'AF8': 807.069580078125, 'TP10': 795.7875366210938}
{'_id': ObjectId('60c019b9d64ad447d49253f1'), 't': '2021-06-08 21:30:33.880992', 'TP9': 834.06591796875, 'AF7': 805.054931640625, 'AF8': 802.2344360351562, 'TP10': 796.996337890625}
{'_id': ObjectId('60c019b9d64ad447d49253f2'), 't': '2021-06-08 21:30:33.880992', 'TP9': 842.12451171875, 'AF7': 809.4871826171875, 'AF8': 805.4578857421875, 'TP10': 801.025634765625}
{'_id': ObjectId('60c019b9d64ad447d49253f3'), 't': '2021-06-08 21:30:33.880992', 'TP9': 880.4029541015625, 'AF7': 806.2637329101562, 'AF8': 805.4578857421875, 'TP10': 795.3846435546875}
{'_id': ObjectId('60c019b9d64ad447d49253f4'), 't': '2021-06-08 21:30:33.881988',

In [13]:
db.processed_0.find_one()

In [13]:
plot_finished_session(list_wavelet_solutions, list_Hjorth_parameters)

ValueError: need at least one array to stack

In [ ]:
#### NOTES Cell

time0=time()

# Using Complex Morlet Wavelets
cmw = lambda t, f, s: (1/np.sqrt(s*np.sqrt(np.pi))) * np.exp(-t**2/(2*s**2)) * np.exp(1j*2*np.pi*f*t)
std_for_gaussian = lambda n, f: n/(2*np.pi*f)

windowLengthSeconds=0.25
fs=256
x = np.linspace(-windowLengthSeconds/2, windowLengthSeconds/2, int(windowLengthSeconds*fs)+1)  
wavelet_freq_peaks = range(5,80,2)
complex_morlet_wavelet = np.vstack([cmw(x, f=f_, s=std_for_gaussian(n=f_/5, f=f_)) for f_ in wavelet_freq_peaks])

print("complex_morlet_wavelet.shape",complex_morlet_wavelet.shape)

for i in range(256*4+1):
    sin_post = {'t':t[i], 'sin_A':sin1[i], 'sin_B':sin2[i], 'sin_C':sin1[i], 'sin_D':sin2[i]}
    db.test_capped3.insert_one(sin_post)
    if i % 64 == 0:
        docs = db.test_capped3.find({}, projection={"sin_A":1,"sin_B":1,"sin_C":1,"sin_D":1, "_id":0})
        X = np.array([[doc['sin_A'], doc['sin_B'], doc['sin_C'], doc['sin_D']] for doc in docs])
        
        wavelet_solution = np.dot(complex_morlet_wavelet, X)   # Convolution-like operation
        
        Hjorth_parameters = {'A': np.var(X, axis=0), 
                             'M': np.sqrt(np.var(np.diff(X, axis=0), axis=0)/np.var(X, axis=0)),
                             'C': np.sqrt(np.var(np.diff(X, n=2, axis=0), axis=0)/np.var(X, axis=0))}
        
        print('sin_A\nsin_B\nsin_C\nsin_D')
        print('6Hz\t\t10Hz\t\t14Hz')
        print('AMPLITUDE\n',np.abs(wavelet_solution))
        print('POWER\n', np.abs(wavelet_solution**2))
        print('PHASE\n', np.angle(wavelet_solution))
        print('HJORTH PARAMETERS',  Hjorth_parameters)
        print('\n')
        
print('Runtime (sec):', time()-time0)

#complex_morlet_wavelet = cmw(x, freq, std_for_gaussian(n=n_wavelet_cycles, f=freq))